# Text Generation via "Retrieval-Augmented Generation"

EXECUTABLE VERSION: [colab](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial7_RAG_Generator.ipynb)


### "Retrieval-Augmented Generation"

In this Tutorial, we want to highlight "Retrieval-Augmented Generation" along with "Dense Dual-Encoder" called Dense Passage Retriever in Haystack codebase. 

Original Abstract: 

_"Large pre-trained language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art results when fine-tuned on downstream NLP tasks. However, their ability to access and precisely manipulate knowledge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-specific architectures. Additionally, providing provenance for their decisions and updating their world knowledge remain open research problems. Pre-trained models with a differentiable access mechanism to explicit non-parametric memory can overcome this issue, but have so far been only investigated for extractive downstream tasks. We explore a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG) -- models which combine pre-trained parametric and non-parametric memory for language generation. We introduce RAG models where the parametric memory is a pre-trained seq2seq model and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. We compare two RAG formulations, one which conditions on the same retrieved passages across the whole generated sequence, the other can use different passages per token. We fine-tune and evaluate our models on a wide range of knowledge-intensive NLP tasks and set the state-of-the-art on three open domain QA tasks, outperforming parametric seq2seq models and task-specific retrieve-and-extract architectures. For language generation tasks, we find that RAG models generate more specific, diverse and factual language than a state-of-the-art parametric-only seq2seq baseline."_

Paper: https://arxiv.org/abs/2005.11401  
Original Code: https://github.com/huggingface/transformers


*Use this* [link](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial7_RAG_Generator.ipynb) *to open the notebook in Google Colab.*


In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack/
!pip install urllib3==1.25.4

In [ ]:
from haystack import Document
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.generator.transformers import RAGenerator
from haystack.retriever.dense import DensePassageRetriever

In [ ]:
# Add documents from which you want generate answers
documents = [
    Document(
        text="""The capital of Germany is the city state of Berlin."""
    ),
    Document(
        text="""Berlin is the capital and largest city of Germany by both area and population.""",
    )
]

In [ ]:
# Initialize FAISS document store to documents and corresponding index for embeddings 
document_store = FAISSDocumentStore()

# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=False,
    embed_title=True,
    remove_sep_tok_from_untitled_passages=True
)

# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    retriever=retriever,
    use_gpu=False,
    top_k_answers=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)

In [ ]:
# Delete existing documents in documents store
document_store.delete_all_documents()
# Write documents to document store
document_store.write_documents(documents)
# Add documents embeddings to index
document_store.update_embeddings(
    retriever=retriever
)

In [ ]:
# Now ask your question and retrieve related documents from retriever
question = "What is capital of the Germany?"
retriever_results = retriever.retrieve(
    query=question
)

In [ ]:
# Now generate answer from question and retrieved documents
predicted_result = generator.predict(
    question=question,
    documents=retriever_results,
    top_k=1
)

# Print you answer
answers = predicted_result["answers"]
for idx, answer in enumerate(answers):
    print(f'Generated answer#{idx + 1} is{answer["answer"]}')